# <font color='red'>Backtest vectorizado para una estrategia 'X' definida.</font>

### Hackeado by J3viton (learning BackTesting 2021).

---

En este notebook vamos a automatizar el backtesting para estrategias. Previamente tenemos que definir **la estrategia** usando el template (import estrategia_XX as rg), donde definimos la clase estrategia que contiene los metodos de analisis, entreda y salida de mercado. Este módulo se programa y adapta para cada estrategia. Luego en el metodo 'analisis' se crea un fichero excel con las metricas y rendimientos para evaluar la estrategia.

Vamos a usar el módulo Pandas, para realizar un backtest vectorizado, es decir calcular el resultado del backtest sin iterar sobre las filas (barras) de la serie temporal.

Posteriormente graficamos y mostramos los parametros generales que nos permiten analizar la estrategia.

#### J3Viton  2021

link a la base:

https://github.com/Python-para-Trading/Webinars-Docs/blob/master/Webinar%202/Webinar%202%20-%20Backtest%20con%20Pandas%20v%202.ipynb.

---

# DEFINICION DEL SISTEMA

Como es usual, importamos los módulos que vamos a necesitar para las distintas tareas, y preparamos Matplotlib para una mejor visualización de los gráficos en el notebook.

Se importa un modulo llamado analisis, en realidad se trata de un archivo .py con algunas funciones de apoyo para el trabajo, debe estar guardado en la misma carpeta que este notebook.
Se importa el módulo 'estrategia_XX', donde se define la estrategia de entreda y salida

Es importante tener clara la lógica del sistema, pues debemos plasmarla de forma sencilla y correcta en el dataframe para realizar el backtest del mismo.


In [1]:
import analisis
import estrategia_XX as rg  #cambiar segun el módulo con la estrategia implementado
import pandas as pd
import datetime as dt
from time import time
import yfinance as yf
import numpy as np

{'is_bot': True, 'id': 1473252352, 'can_join_groups': True, 'supports_inline_queries': False, 'first_name': 'vital_bot', 'username': 'vital_quant_bot', 'can_read_all_group_messages': False}


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 18, 8
plt.style.use('ggplot')

from IPython.display import display, Markdown

***0.- RECOGIDA DE DATOS INICIALES***
Introducimos el instrumento y las fechas


In [3]:
instrumento_ = 'ntgy.MC'   # ticker del valor en yahooFinance

Fecha_Cominezo_Backtesting = dt.datetime(2008,1,2)
Fecha_Final_Backtesting    = dt.datetime(2021,11,18)




***1.- Rango fechas a analizar***

Definimos el rango global de datos historicos que vamos a evaluar.
Definimos la ventana que vamos a ir desplazando por todo el espectro para ir analilazando como se comporta la estrategia. La funion 'analisis' nos vale para tiempo real y para backtesting, para back le pasamos la ventana como si la fecha de fin de la ventana fuera la fecha de hoy.
Tener en cuenta que la 'ventana' tiene uqe tener una anchura que nos permita hacer los calculos en rolling (ejemplo EMA 200)

In [4]:
# Rango completo para backTesting
#start2 =dt.datetime(2008,1,2)
start2= Fecha_Cominezo_Backtesting 
#end2   =dt.datetime(2021,11,18)
end2= Fecha_Final_Backtesting 
start_G= start2.strftime("%Y-%m-%d")
end_G  =   end2.strftime("%Y-%m-%d")
TOTAL_len= (end2-start2).days
print('Tamaño timeseries global a analizar:  ', TOTAL_len, 'sesiones')

#ventana de analisis 200 sesiones
startWindow2 =dt.datetime(2008,1,5)
endWindow2   =startWindow2 + dt.timedelta(days=1000) #ventana grande para que se puedan hacer los calculos de EMA200
startWindow= startWindow2.strftime("%Y-%m-%d")
endWindow  =   endWindow2.strftime("%Y-%m-%d")
window_len= (endWindow2-startWindow2).days
print('Tamaño de la ventana a analizar paso a paso:  ', window_len, 'sesiones')

Tamaño timeseries global a analizar:   5069 sesiones
Tamaño de la ventana a analizar paso a paso:   1000 sesiones


In [5]:
 #dff = pd.DataFrame(columns=('Close','Volume', 'Senal', 'Dif_Close', 'Retornos','Capital'))

***2.- Descargamos los datos para el marco Global***

In [6]:
instrumento = instrumento_  # 'rovi.mc'  #Vamos título a título. Mejora: Conjunto de títulos

In [7]:
dff = yf.download(instrumento, start_G,end_G)

[*********************100%***********************]  1 of 1 completed


In [8]:
dff.dropna(inplace=True)  
dff.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,33.834824,34.567215,33.417530,33.570820,14.141039,888723
2008-01-03,33.221657,33.971081,32.889526,33.792240,14.234302,1708698
2008-01-04,33.638950,33.638950,31.901648,32.216747,13.570661,5944283
2008-01-07,32.123070,33.119465,31.390676,32.378555,13.638817,1101285
2008-01-08,32.216747,32.625523,31.910164,32.174168,13.707756,1307534


***3.- Creamos la clase strategy que tiene toda la lógica***

In [9]:
regreMedia= rg.StrategyClass()    #Creamos la clase

***4.- Recorremos el dataframe con valores buscando las señales de la estrategia***

In [10]:
dff.index

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2021-11-04', '2021-11-05', '2021-11-08', '2021-11-09',
               '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15',
               '2021-11-16', '2021-11-17'],
              dtype='datetime64[ns]', name='Date', length=3547, freq=None)

## Proceso de backTesting ##

En este 'for' vamos recorriendo la muestra del historico de datos (TOTAL), desplazando una sesión hacia el futuro 
en cada iteracion. Vamos pasando la movilola del pasado sesión a sesión por el análisis descrito en Strategy class.

Ouput:
El sistema registra los siguientes parámetros:

.-
.-
.-


In [11]:
startWindow

'2008-01-05'

In [12]:
tiempo_inicial = time()   # Tomamos tiempos para ver cuanto tarda en hacer la estrategia

In [13]:
dfe = pd.DataFrame({'A' : []})   #df empty

***En este 'for' desplazamos la ventana sesión a sesión a lo largo de todo el rango de fechas. Dejamos que las funciones de la clase estrategia hagan el trabajo de comprar//vender y anotar***


In [ ]:
#Para pruebas
#TOTAL_len =100

for i in range(TOTAL_len):
    endWindow3   =endWindow2 + dt.timedelta(days=i) 
    endWindow    =endWindow3.strftime("%Y-%m-%d")
    print ('end date:', endWindow)
    
    if(endWindow in dff.index):
        df_aux= dff.loc[startWindow:endWindow]       #voy pasando los datos desplazando la ventana
        
        recogo = regreMedia.analisis(instrumento, startWindow, endWindow, df_aux) #Llamada a la clase estrategia. LA CLAVE DE TODO!!!
        
        print ('................................................Analizando, muestra', i, 'de', TOTAL_len, 'fecha', endWindow)
        
        """
        Tratamiento de los returns, mejor en las funciones de analisis que hacer aquí.
        if(recogo['senal'] == 1 ):
            dff.loc[[endWindow],['Senal']]= 1
        elif(recogo['senal'] == 0 ):
            dff.loc[[endWindow],['Senal']]= 0
        elif(recogo['senal'] == -1 ):
            dff.loc[[endWindow],['Senal']]= -1            
        elif(recogo['senal'] == 503 ):
            dff.loc[[endWindow],['Senal']]= 0          
        """
            
    else:
        print('..............Día sin sesión, next please')

        

end date: 2010-10-01
 datos desde archivo
Coefficients: 
 [-0.03249297]
Independent term: 
 29.685309031296136
Coefficients: 
 [-0.01648788]
Independent term: 
 14.623269381736641
Coefficients: 
 [-0.01021156]
Independent term: 
 14.904382034097871
  instrumento  long_short_out        date     precio  beneficio  stoploss
0      IBE.MC               1  2011-08-10   4.776000        NaN       NaN
1        MSFT               0  2014-01-08  35.759998   1.350209      99.0
2      san.mc               1  2021-07-08   3.057500   0.532204      99.0
3      ibe.mc               0  2021-06-25  10.355000   0.797811      99.0
4      REE.MC               0  2019-07-08  17.695000   1.585407      99.0
................................................Analizando, muestra 0 de 5069 fecha 2010-10-01
end date: 2010-10-02
..............Día sin sesión, next please
end date: 2010-10-03
..............Día sin sesión, next please
end date: 2010-10-04
 datos desde archivo
Coefficients: 
 [-0.03245206]
Independent te

C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:870: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")
C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:870: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence thi

 datos desde archivo
Coefficients: 
 [-0.03240986]
Independent term: 
 29.666013662340987
Coefficients: 
 [-0.01679005]
Independent term: 
 14.612460426074355
Coefficients: 
 [-0.01034055]
Independent term: 
 14.898677898392465
  instrumento  long_short_out        date     precio  beneficio  stoploss
0      IBE.MC               1  2011-08-10   4.776000        NaN       NaN
1        MSFT               0  2014-01-08  35.759998   1.350209      99.0
2      san.mc               1  2021-07-08   3.057500   0.532204      99.0
3      ibe.mc               0  2021-06-25  10.355000   0.797811      99.0
4      REE.MC               0  2019-07-08  17.695000   1.585407      99.0
................................................Analizando, muestra 4 de 5069 fecha 2010-10-05
end date: 2010-10-06
 datos desde archivo
Coefficients: 
 [-0.03236898]
Independent term: 
 29.65650174296215
Coefficients: 
 [-0.0169013]
Independent term: 
 14.601479827254567
Coefficients: 
 [-0.01040583]
Independent term: 
 14.89

C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:870: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")
C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:870: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence thi

 datos desde archivo
Coefficients: 
 [-0.03232653]
Independent term: 
 29.64661042946117
Coefficients: 
 [-0.01696241]
Independent term: 
 14.585309676839344
Coefficients: 
 [-0.01047107]
Independent term: 
 14.892632036721292
  instrumento  long_short_out        date     precio  beneficio  stoploss
0      IBE.MC               1  2011-08-10   4.776000        NaN       NaN
1        MSFT               0  2014-01-08  35.759998   1.350209      99.0
2      san.mc               1  2021-07-08   3.057500   0.532204      99.0
3      ibe.mc               0  2021-06-25  10.355000   0.797811      99.0
4      REE.MC               0  2019-07-08  17.695000   1.585407      99.0
................................................Analizando, muestra 6 de 5069 fecha 2010-10-07
end date: 2010-10-08
 datos desde archivo
Coefficients: 
 [-0.03228502]
Independent term: 
 29.636924628551917
Coefficients: 
 [-0.01702124]
Independent term: 
 14.568263421414503
Coefficients: 
 [-0.01053625]
Independent term: 
 14.8

...

  instrumento  long_short_out        date     precio  beneficio  stoploss
0      IBE.MC               1  2011-08-10   4.776000        NaN       NaN
1        MSFT               0  2014-01-08  35.759998   1.350209      99.0
2      san.mc               1  2021-07-08   3.057500   0.532204      99.0
3      ibe.mc               0  2021-06-25  10.355000   0.797811      99.0
4      REE.MC               0  2019-07-08  17.695000   1.585407      99.0
5     ntgy.MC               0  2017-07-28  19.370001   2.478322      99.0
................................................Analizando, muestra 2798 de 5069 fecha 2018-05-30
end date: 2018-05-31
 datos desde archivo
Coefficients: 
 [0.00074031]
Independent term: 
 16.29956208463489
Coefficients: 
 [0.00808062]
Independent term: 
 18.650928426429406
Coefficients: 
 [-0.00195841]
Independent term: 
 19.62160678437712


C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:870: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")


  instrumento  long_short_out        date     precio  beneficio  stoploss
0      IBE.MC               1  2011-08-10   4.776000        NaN       NaN
1        MSFT               0  2014-01-08  35.759998   1.350209      99.0
2      san.mc               1  2021-07-08   3.057500   0.532204      99.0
3      ibe.mc               0  2021-06-25  10.355000   0.797811      99.0
4      REE.MC               0  2019-07-08  17.695000   1.585407      99.0
5     ntgy.MC               0  2017-07-28  19.370001   2.478322      99.0
................................................Analizando, muestra 2799 de 5069 fecha 2018-05-31
end date: 2018-06-01
 datos desde archivo
Coefficients: 
 [0.0007426]
Independent term: 
 16.2975364011831
Coefficients: 
 [0.00844697]
Independent term: 
 18.6173015600176
Coefficients: 
 [-0.00185487]
Independent term: 
 19.61161369756758


C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:870: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")


  instrumento  long_short_out        date     precio  beneficio  stoploss
0      IBE.MC               1  2011-08-10   4.776000        NaN       NaN
1        MSFT               0  2014-01-08  35.759998   1.350209      99.0
2      san.mc               1  2021-07-08   3.057500   0.532204      99.0
3      ibe.mc               0  2021-06-25  10.355000   0.797811      99.0
4      REE.MC               0  2019-07-08  17.695000   1.585407      99.0
5     ntgy.MC               0  2017-07-28  19.370001   2.478322      99.0
................................................Analizando, muestra 2800 de 5069 fecha 2018-06-01
end date: 2018-06-02
..............Día sin sesión, next please
end date: 2018-06-03
..............Día sin sesión, next please
end date: 2018-06-04
 datos desde archivo


***Recogemos los datos de las entradas que ha realizado la Estrategia***

In [ ]:
data=rg.StrategyClass.dfLog
#data.dropna(inplace=True) 

In [ ]:
data.tail()

In [ ]:
data['Senal'].plot(title = 'Señales de compra de la estrategia ' )
# Ploteando una parte
#data.loc['2010':'2022','Senal'].plot(title = 'Señales Regresión a la media ' +instrumento,xlim=('2010','2022'))

In [ ]:
tiempo_final = time() 
 
tiempo_ejecucion = tiempo_final - tiempo_inicial
 
print ('El tiempo de ejecucion fue:',tiempo_ejecucion,"segundos") #En segundos

In [ ]:
data.loc['2011-01-04':'2019-11-04']

In [ ]:
#Guardamos el resultado del analisis en un pickle
#dff.to_pickle("./primerBack_IBE.pkl")

***3.- Ingeniería de datos para calcular la bondad de la estrategia***

In [ ]:

data['Dif_Close'] = data.Price.pct_change()
data['Retornos'] = data.Dif_Close * data.Senal.shift(1)   
data['Capital'] = (data.Retornos + 1).cumprod() * 100
    
rg.StrategyClass.dfLog=data  #Ojo a esta liena, no me cuadra.

#quant_j.salvarExcel(StrategyClass.dfLog, "log"+instrumento)   
#data.to_pickle('almacen')    #df = pd.read_pickle(file_name)

>Para calcular los retornos del sistema, calculamos la diferencia relativativa entre el precio de cierre y el del día anterior. Para posteriormente multiplicarlo por la señal que del día anterior que nos indicaba la posición a tomar (si estaba comprado, sumo beneficio).

In [ ]:
data.tail()

In [ ]:
data[60:]

>El capital, por comodidad, lo calcularemos en base 100, es decir como si iniciaramos la inversión con 100 unidades monetarias. Para su calculo arrastramos el producto acumulado de los retornos mas 1, multiplicados como dijimos por 100.

Pero para observar gráficamente la evolución del sistema necesitamos un gráfico con mas información. Para ello definimos una función que nos mostrará un gráfico con la evolución de nuestra estrategia, comparandola con la del activo. En un subgráfico se visualizará el drawdown del sistema, comparado de nuevo con el del activo. Y por útlimo las posiciones que toma el sistema.

In [ ]:
def grafico (df):
    estudio = df.copy()
    
    DD_bh, maxDD, maxDD_ini, maxDD_fin = analisis.DrawDown(estudio.Dif_Close[60:], info = False) 
    
    DD, maxDD, maxDD_ini, maxDD_fin = analisis.DrawDown(estudio.Retornos.fillna(0), info = False) 

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(18, 12), gridspec_kw = {'height_ratios':[3, 1, 1]})
    fig.suptitle('Estrategia vs B & H', fontsize=20)

    #Primer plot analiza la evolcuion de una inversion de 100€ en buy&hold y la estrategia)
    ax1.plot(estudio.Capital)
    ax1.plot((estudio.Price) * (100/estudio.Price[1]))
    ax1.set_title('Capital')
    ax1.legend(['Estrategia','Buy & Hold'])

    ax2.plot(DD*100, c='0.5')
    ax2.plot(DD_bh*100, c='y')
    ax2.fill_between(DD.index, 0, DD*100, color='0.7')
    ax2.set_title('Drawdown')
    ax2.legend(['Estrategia','Buy & Hold'])

    ax3.plot(estudio.Senal, c='orange')
    #ax3.fill_between(estudio.index, 0, estudio.Senal*100, color='orange')
    ax3.set_title('Posición')

    plt.show()
    return

In [ ]:
grafico(data)

---

In [ ]:
#data[55:70]

---

# ANALISIS DEL SISTEMA

In [ ]:
Beneficio_Bruto = data.Capital[-1] - data.Capital[1]
print ('Beneficio Bruto : {:.2f} €  con una inverison de 100€'.format(Beneficio_Bruto))

In [ ]:
#Rentabilidad anual compuesta. Calculamos lo que la inversion se ha incrementado cada año con el interes compuesto
CAGR = analisis.CAGR(data.Retornos)
print ('CAGR : {:.2f}%'.format(CAGR * 100))

In [ ]:
#Ratio calculado restando una rentabilidad segura a nuestra rentabilidad de estrategia y dividido por el riego (=volatilidad)
data.Retornos=data.Retornos.replace(0,np.e)  #Quito los cero (no sepuede hacer logariotmo de cero), pero no se que poner
Sharpe = CAGR / (np.log(data.Retornos + 1).std(skipna=True) * 252 ** 0.5)
print ('Sharpe : {:.3f}'.format(CAGR ))

In [ ]:
# Actualizo la excel con los economic
regreMedia.analisisEconomics(instrumento)

---


---